In [1]:
# To use it:
# https://github.com/charles9n/bert-sklearn

In [2]:
import pickle
from bert_sklearn import BertClassifier
from bert_sklearn import load_model
import pandas as pd
import pathlib
from tools import processing

import spacy

nlp = spacy.load("en_core_web_sm")

model = BertClassifier()    

Building sklearn text classifier...


#### Load data

In [3]:
corpus = "stories/"
dataset_df = pd.read_pickle("../data/" + corpus + "train.pkl")
dataset_df['both_masked'] = dataset_df.apply(lambda row: processing.determine_context("maskedSentWithDet", row, "both"), axis=1)
dataset_df['both_unmasked'] = dataset_df.apply(lambda row: processing.determine_context("currentSentence", row, "both"), axis=1)

#### Train BERT classifiers

In [4]:
dFolders = {"targetWithoutDet": "../models/classifiers/" + corpus + "targetWithoutDet_without_prp/",
            "context3wmasked": "../models/classifiers/" + corpus + "context3wmasked_without_prp/",
            "context3w": "../models/classifiers/" + corpus + "context3w_without_prp/"}
# ,
#             "both_masked": "../models/classifiers/" + corpus + "both_masked_with_prp/",
#             "both_unmasked": "../models/classifiers/" + corpus + "both_unmasked_with_prp/",
#             "currentSentence": "../models/classifiers/" + corpus + "currentSentence_with_prp/",
#             "maskedSentence": "../models/classifiers/" + corpus + "maskedSentence_with_prp/"}

for col in ["targetWithoutDet", "context3wmasked", "context3w"]: #, "both_masked", "both_unmasked", "currentSentence", "maskedSentWithDet"]:
    dataset_df = dataset_df[dataset_df["targetIsPRP"] == False]
    print(col)
    if not pathlib.Path(dFolders[col] + "/bert.bin").exists():
        pathlib.Path(dFolders[col]).mkdir(parents=True, exist_ok=True)

        X = dataset_df[col].tolist()
        y = dataset_df["animated"].tolist()

        print (len(X),len(y))

        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # finetune model
        model.fit(X_train, y_train)

        # make predictions
        y_pred = model.predict(X_test)

        from sklearn.metrics import precision_recall_fscore_support

        precision_recall_fscore_support(y_pred, y_test, average='macro')

        # save model to disk
        savefile= dFolders[col] + "/bert.bin"
        model.save(savefile)
        print("DONE")
        print()

targetWithoutDet
8457 8457
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 5328, validation data size: 591


Validating: 100%|██████████| 74/74 [02:19<00:00,  1.87s/it]

Epoch 1, Train loss: 0.3402, Val loss: 0.2074, Val accy: 93.23%



Validating: 100%|██████████| 74/74 [02:20<00:00,  1.81s/it]

Epoch 2, Train loss: 0.1583, Val loss: 0.1821, Val accy: 94.08%


...

Epoch 2, Train loss: 0.1878, Val loss: 0.2363, Val accy: 91.37%



Validating: 100%|██████████| 74/74 [01:34<00:00,  1.19s/it]

Epoch 3, Train loss: 0.1172, Val loss: 0.2457, Val accy: 91.71%



Predicting: 100%|██████████| 318/318 [06:39<00:00,  1.01s/it]


DONE

